In [ ]:
import warnings, os
import datetime as dt
from dateutil.relativedelta import relativedelta as rtd
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin([*SYMBOLS]) & Raw['tf'].isin([*TFRAMES])] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['Hilo']     = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 

        Df = Calc[Calc['Time'] == dt.time(HR,00)]

        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)

In [24]:
STATS(Calc, Col='ID Change')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,-7,479,-2355,-290,10,280,2045
1,11:00:00,1240,5,758,-2640,-496,-8,500,4690
2,12:00:00,1240,-26,914,-3630,-630,-25,561,4295
3,13:00:00,1240,-14,1001,-3690,-685,-22,690,3130
4,14:00:00,1245,-13,1038,-3470,-745,5,705,3130
5,15:00:00,1245,-13,1105,-3835,-745,20,740,5555
6,16:00:00,1245,-21,1165,-4520,-775,20,790,4130
7,17:00:00,1245,-16,1220,-4030,-810,35,795,4890
8,18:00:00,1245,-29,1279,-4010,-845,25,785,4610
9,19:00:00,682,-6,1300,-5150,-844,62,829,4535


In [25]:
STATS(Calc, Col='ID Chg Abs')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,364,312,0,135,290,515,2355
1,11:00:00,1240,595,470,0,249,500,835,4690
2,12:00:00,1240,718,565,0,284,618,1010,4295
3,13:00:00,1240,800,601,0,330,685,1140,3690
4,14:00:00,1245,831,622,0,320,720,1200,3470
5,15:00:00,1245,881,666,0,360,745,1275,5555
6,16:00:00,1245,921,714,0,350,785,1320,4520
7,17:00:00,1245,967,744,0,410,805,1355,4890
8,18:00:00,1245,1004,792,0,415,825,1430,4610
9,19:00:00,682,1014,813,0,410,840,1400,5150


In [26]:
STATS(Calc, Col='ID Chg Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,636,348,301,0,115,270,498,2045
1,11:00:00,614,605,479,0,241,502,855,4690
2,12:00:00,608,706,556,0,275,592,996,4295
3,13:00:00,610,799,595,0,320,700,1149,3130
4,14:00:00,629,809,613,0,300,695,1200,3130
5,15:00:00,632,856,654,0,340,730,1265,5555
6,16:00:00,633,885,682,0,325,775,1255,4130
7,17:00:00,639,927,722,0,370,780,1318,4890
8,18:00:00,640,949,767,0,370,755,1346,4610
9,19:00:00,360,955,775,0,382,795,1320,4535


In [27]:
STATS(Calc, Col='ID Chg Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,604,-381,323,-2355,-530,-300,-150,-5
1,11:00:00,626,-584,462,-2640,-804,-492,-250,-5
2,12:00:00,632,-730,575,-3630,-1015,-625,-295,-5
3,13:00:00,630,-801,607,-3690,-1119,-682,-335,-5
4,14:00:00,616,-852,630,-3470,-1195,-750,-345,-5
5,15:00:00,613,-908,678,-3835,-1305,-770,-395,-5
6,16:00:00,612,-957,744,-4520,-1388,-795,-375,-5
7,17:00:00,606,-1009,764,-4030,-1445,-830,-446,-5
8,18:00:00,605,-1062,815,-4010,-1520,-885,-475,-5
9,19:00:00,322,-1081,850,-5150,-1449,-882,-466,-10


In [28]:
STATS(Calc, Col='ID Hilo')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,745,332,215,505,670,905,2965
1,11:00:00,1240,1176,464,340,865,1090,1380,5635
2,12:00:00,1240,1417,551,470,1045,1305,1675,5950
3,13:00:00,1240,1561,594,510,1160,1462,1820,5950
4,14:00:00,1245,1641,616,530,1210,1525,1935,5950
5,15:00:00,1245,1737,672,535,1275,1615,2060,7400
6,16:00:00,1245,1826,708,535,1345,1685,2180,7400
7,17:00:00,1245,1904,763,535,1385,1745,2270,7400
8,18:00:00,1245,1959,798,535,1420,1785,2350,7400
9,19:00:00,682,1920,852,535,1335,1702,2329,7400


In [29]:
STATS(Calc, Col='ID HL Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,636,736,317,215,500,670,895,2355
1,11:00:00,614,1185,465,340,870,1102,1398,5635
2,12:00:00,608,1408,544,470,1040,1308,1670,5950
3,13:00:00,610,1548,598,510,1155,1455,1804,5950
4,14:00:00,629,1610,599,530,1170,1525,1895,5950
5,15:00:00,632,1703,680,535,1234,1595,2015,7400
6,16:00:00,633,1782,698,535,1325,1645,2120,7400
7,17:00:00,639,1852,756,535,1365,1695,2192,7400
8,18:00:00,640,1896,782,535,1395,1725,2255,7400
9,19:00:00,360,1876,837,535,1274,1660,2274,7400


In [30]:
STATS(Calc, Col='ID HL Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,604,-754,347,-2965,-911,-670,-510,-275
1,11:00:00,626,-1168,463,-3345,-1365,-1085,-856,-370
2,12:00:00,632,-1425,558,-4390,-1680,-1305,-1045,-490
3,13:00:00,630,-1575,590,-4715,-1835,-1465,-1160,-590
4,14:00:00,616,-1672,632,-4800,-1978,-1540,-1230,-610
5,15:00:00,613,-1771,663,-4715,-2095,-1635,-1305,-665
6,16:00:00,612,-1871,716,-5205,-2226,-1728,-1360,-665
7,17:00:00,606,-1959,766,-5225,-2349,-1800,-1411,-665
8,18:00:00,605,-2026,810,-5225,-2425,-1825,-1450,-610
9,19:00:00,322,-1970,868,-5720,-2358,-1765,-1378,-610


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc


def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 
        Df = Calc[Calc['Time'] == dt.time(HR,00)]
        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)